In [ ]:
import torch
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from sklearn.model_selection import train_test_split

In [ ]:
size1 = 128
size2 = 128
train_size = 0.3

In [ ]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.random.seed = random_seed

In [ ]:
import torch 
import pandas as pd
from PIL import Image 
from os import listdir
from os.path import isfile, join
import torchvision.transforms as transforms 
import pandas as pd
import math

train_csv = pd.read_csv("train_labels.csv")

path = 'images/images/train/'

train_features = []
train_targets = []

transform = transforms.Compose([ 
    transforms.PILToTensor(),
    transforms.Resize((size1, size2)) 
]) 

for file in listdir(path):
    filename = join(path, file)
    if isfile(filename):
        image = Image.open(filename)
        width, height = image.size
        img_tensor = transform(image) 
        train_features.append(img_tensor)
        train_targets.append(train_csv.loc[train_csv['file_name'] == file].drop('file_name', axis = 1).values[0])
        for i in range(0, len(train_targets[-1]),2):
            train_targets[-1][i] = train_targets[-1][i]*(size1/width)
            train_targets[-1][i+1] = train_targets[-1][i+1]*(size2/height)

In [ ]:
dataset = torch.utils.data.TensorDataset(torch.stack(train_features).type('torch.FloatTensor'), torch.FloatTensor(train_targets))

In [ ]:
torch.save(dataset, 'datasets/train.pt')